In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install pyarrow
! pip install PyPDF2
!apt-get install poppler-utils 
!pip install sentencepiece
!pip install transformers
import pandas as pd
import json
from transformers import BertForTokenClassification, BertConfig
from transformers import get_scheduler
from tqdm.auto import tqdm
from transformers import AdamW
import torch
import pytesseract #python version for tesseract 
from pdf2image import convert_from_path 
#Warnings
import PyPDF2
import warnings
warnings.filterwarnings("ignore")
import os
import re
import logging
import numpy as np
from tqdm import trange
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
#from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix
import argparse
import torch
from transformers import BertForTokenClassification, BertTokenizerFast, DistilBertTokenizer, TFDistilBertForTokenClassification,DistilBertTokenizer, T5Config,DistilBertForTokenClassification
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam



Lets download tesseract OCR for character recognisation in resumes

In [3]:
class Tesseracttext:
    def __init__(self,path):
        self.path=path
    def _pdftoimg(self):
        pages=convert_from_path(self.path,500)
        
        return pages
    def _imgtotx(self):
        pages=self._pdftoimg()
        for num,page in enumerate(pages):
            try:
                
                   text = pytesseract.image_to_string(page,lang='eng',timeout=60)
                   print(text)
            except RuntimeError as timeout_error:
    # Tesseract processing is terminated
                   pass
            #image_to_boxes
        
        
s=Tesseracttext('../input/my-resume/vivek_l_alex_resume.pdf')
s._imgtotx()  

### NER dataset preprocessing

In [ ]:
tags_vals = ["UNKNOWN", "O", "Name", "Degree", "Skills", "College Name", "Email Address",
             "Designation", "Companies worked at", "Graduation Year", "Years of Experience", "Location"]

tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals)}

MAX_LEN = 500
EPOCHS = 10
MAX_GRAD_NORM = 1.0
MODEL_NAME = 'bert-base-uncased'
TOKENIZER = BertTokenizerFast('/content/Resume-NER/vocab/vocab.txt', lowercase=True)
def convert_goldparse(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines = []
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content'].replace("\n", " ")
            entities = []
            data_annotations = data['annotation']
            if data_annotations is not None:
                for annotation in data_annotations:
                    point = annotation['points'][0]
                    labels = annotation['label']
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        point_start = point['start']
                        point_end = point['end']
                        point_text = point['text']

                        lstrip_diff = len(point_text) - \
                            len(point_text.lstrip())
                        rstrip_diff = len(point_text) - \
                            len(point_text.rstrip())
                        if lstrip_diff != 0:
                            point_start = point_start + lstrip_diff
                        if rstrip_diff != 0:
                            point_end = point_end - rstrip_diff
                        entities.append((point_start, point_end + 1, label))
            training_data.append((text, {"entities": entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " +
                          dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

sd= convert_goldparse('/content/Resume-NER/data/Resumes.json')
sd1=trim_entity_spans(sd)

In [ ]:
def id_maker(input,tokenizer, tag2idx, max_len=500):
    tokenizer=tokenizer(input[0],padding='max_length',return_offsets_mapping=True, max_length=max_len, truncation=True, return_tensors="pt")
    labels=input[1]['entities']
    labels.reverse()
    orig_label=[]
    lab=[]
   #padding_length = max_len - len()
    def offs(off,labels):
                  
    
       if (int(off[0]) == 0 and int(off[1]) == 0):
         return 'UNKNOWN'
      
      
       for label in labels:
        
             if (int(off[1])<=label[1] and int(off[0])>=label[0]) :
                return label[2]
      
       return 'O'
    for off in tokenizer['offset_mapping'][0]:
    
        a=offs(off,labels)
        orig_label.append(a)
        lab.append(tag2idx[a])
    return tokenizer,orig_label,lab   


In [ ]:
labels=[]
for ai in range(len(sd1)) :
  tokenizer,orig_label,lab= id_maker(sd1[ai],TOKENIZER,tag2idx)
  labels.append(lab)

train_x=[]
for i in range(len(sd1)):
  train_x.append(sd1[i][0])

In [ ]:
from torch.utils.data import Dataset
import torch as th

class GSMDataset(th.utils.data.Dataset):
    def __init__(self, tokenizer,x_train,y_train):
        self.lens=len(x_train)
        self.sent = x_train
        self.ent = y_train
        self.sent = tokenizer(self.sent,padding='max_length',return_offsets_mapping=True, max_length=500, truncation=True, return_tensors="pt")
        self.ent =  [-100 if x==0 else x for x in self.ent]
        
    def __len__(self):
        return self.lens

    def __getitem__(self, idx):
        sent_tokens = self.sent["input_ids"][idx]
        sent_att=self.sent["attention_mask"][idx]
        ent_tokens = self.ent[idx]
        ent_tokens = [-100 if x==0 else x for x in ent_tokens] 
        sn_tokens = th.tensor(sent_tokens)
        sn_att = th.tensor(sent_att)
        en_tokens=th.tensor(ent_tokens)
        
        return dict(input_ids= sn_tokens, attention_mask=sn_att,labels=en_tokens)

tr=GSMDataset(TOKENIZER,train_x,labels)
#ts=GSMDataset(TOKENIZER,x_test,y_test)

### Model Training

In [ ]:
config = BertConfig.from_pretrained("bert-base-uncased")
model = BertForTokenClassification.from_pretrained( "bert-base-uncased",num_labels=len(tag2idx))
model.train()
train_loader = DataLoader(tr, batch_size=16, shuffle=True)
optim = AdamW(model.parameters(), lr=1e-5)
num_epochs = 20
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optim,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

pbar = tqdm(range(num_training_steps))
for epoch in range(num_epochs):
        for batch in train_loader:
            optim.zero_grad()
            b_input_ids,b_attention_mask,b_labels= batch.items()
            outputs = model(input_ids=batch["input_ids"],attention_mask=batch["attention_mask"], labels=torch.tensor(batch["labels"]))
            loss = outputs[0]
            loss.backward()
            optim.step()
            lr_scheduler.step()
            pbar.update(1)
            pbar.set_description(f"train_loss: {loss.item():.5f}")
            model.save_pretrained("model_ckpts/")

In [ ]:

model_path = "/content/model_ckpts"
model =  BertForTokenClassification.from_pretrained(model_path,num_labels=len(tag2idx)
resume="vivek l alex Web Developer Final year B.Tech student with 3+ years of experience in building web applications, college projects, freelancing, and contributing toOpen Source Softwares. JSS Boys Hostel, C Block,Sector - 62, Noida.(+91) 950000000 vivek61@gmail.com  EXPERIENCE 1mg —  Software Engineer Intern Jan 2020 - Present Avanti Learning Centres — Data Visualization Analyst Jul 2019 - Aug 2019 Responsibilities Task 1 - Report card generatorT ask 2 - Automate student summary generation in bulkMotion Invite — UI/UX & Front-end Developer Jan 2018 - Mar 2018 ResponsibilitiesTask 1 - Redesign the existed websiteTask 2 - Develop the redesigned versionEdcams — Front-end Developer Nov 2017 - Jan 2018 Responsibilities Task 1 - Fix the bugsTask 2 - Develop a new sign up portal EDUCATION cochin college of ebgineering , Noida — BTech 2016 - Present  ●  Average Percentage - 74.20% ● Coursework included○ Algorithms○ Data Structures○ SQL based DBMS○ Turing Machines○ OS Concepts● Have taken hands-on workshops on○ C/C++ Programming○ Designing (UI/UX & Photoshop)○ Web Development (HTML & CSS)SKILLS Frontend:ReactJS, Gatsby, jQuery, JavaScript,HTML, CSS, Materialize, Bootstrap,BulmaBackend: FlaskData Visualization: D3, MatplotlibData Analysis: PandasCloud Suite: Google Cloud PlatformProgramming Languages: C++, C,PythonDatabase: Oracle SQLData Structures and Algorithms in C++ Agile/Scrum and Design thinking approaches ACHIEVEMENTS Special mention at DevFestNoida-18 for FitMo (React-basednative application gamifying themotivation towards fitness)Among top 10 national finalists atSocial Track organized byECell-IITKSecond runner-up at B-Plancompetition organized by ATTACNG")

inputs = TOKENIZER(resume,padding='max_length',max_length=500, truncation=True, return_tensors="pt")
inputs_off = TOKENIZER(resume,padding='max_length',return_offsets_mapping=True, max_length=500, truncation=True, return_tensors="pt")
#predicitng

with torch.no_grad():
    logits = model(**inputs).logits
predicted_token_class_ids = logits.argmax(-1)

entities=predicted_token_class_ids
entities=[idx2tag[t.item()] for t in predicted_token_class_ids[0]]
tokenized_text=[resume[int(of[0]):int(of[1])] for of in list(inputs_off.offset_mapping[0])]
location_entity=list(inputs_off.offset_mapping[0])



### JSON File Creator

In [ ]:


tuple_entity_words=list(zip(entities,location_entity))
temp_list=[]
temp_list_2=[]
list_similar_entity=[]
list_similar_entity_loc=[]
#converting offset  maps to text
for i in  range(len(tuple_entity_words)):
  if i != range(len(tuple_entity_words))[-1]:

     if tuple_entity_words[i][0] == tuple_entity_words[i+1][0]:
       temp_list.append( tuple_entity_words[i][0])
       temp_list_2.append(tuple_entity_words[i][1])
     else:
       temp_list.append(tuple_entity_words[i][0])
       temp_list_2.append(list(tuple_entity_words[i][1])) 
       list_similar_entity.append(temp_list)
       list_similar_entity_loc.append(temp_list_2)
       temp_list=[]
       temp_list_2=[]
    


ent_list=[]
#json  making 
for i in range(len(list_similar_entity_loc)):
  ent=list_similar_entity[i][0]

  start=list_similar_entity_loc[i][0][0]
  end=list_similar_entity_loc[i][-1][1]
  text=resume[start:end]
  if ent !="O" and text != '':
     ent_list.append({ent:text})

out_file = open("resume.json", "w") 
json.dump(ent_list, out_file, indent = 6) 
out_file.close() 
